`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
 

# Лабораторная работа №6: Машины опорных векторов. 


В практических примерах ниже показано:    
* как классифицировать данные с помощью модели SVM
* как использовать конвейеры для подгонки модели и применения её к новым данным

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.  


## Данные

1.   `fixed_acidity` – постоянная кислотность;
2.   `volatile_acidity` – переменная кислотность;
3.   `citric_acid` – содержание лимонной кислоты;
4.   `residual_sugar` - остаточный сахар;
5.   `chlorides` – содержание хлоридов;
6.   `free_sulfur_dioxide` – содержание диоксида серы в свободном виде;
7.   `total_sulfur_dioxide` – общее содержание диоксида серы;
8.   `density` – плотность;
9.   `pH` – кислотность;
10.  `sulphates` – содержание сульфатов;
11.  `alcohol` – содержание алкоголя;
12.  `quality` – балльная оценка качества вина, от 0 до 10;
13.  `Y` – целевая переменная: 1 = высокое качество (quality > 5), 0 = низкое (quality <= 5).

# Указания к выполнению


## Загружаем пакеты

In [1]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency, t
#  для таймера
import time

# загрузка пакетов: модели ------------------------------------------------- 
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression 
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split 
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report

# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets

# загрузка пакетов: модели -------------------------------------------------
#  дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree


# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier

# для теста хи-квадрат
import scipy

In [2]:
# константы
#  ядро для генератора случайных чисел
my_seed = 24
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [3]:
# загружаем таблицу и превращаем её во фрейм
DF_raw = pd.read_csv('https://raw.githubusercontent.com/ania607/ML/main/data/winequality-red_for_lab.csv')

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (1599, 13)


In [4]:
DF_raw.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Y
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [5]:
# типы столбцов
DF_raw.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
Y                         int64
dtype: object

In [6]:
DF_raw.isna().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Y                       0
dtype: int64

In [7]:
DF = DF_raw.sample(frac = 0.85, random_state = my_seed)
DF_predict = DF_raw.drop(DF.index)

In [8]:
DF.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Y
count,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000
mean,8.334952,0.527531,0.271847,2.519536,0.087394,15.786240,45.640177,0.996746,3.310545,0.657756,10.436019,5.637233,0.538631
std,1.754752,0.179187,0.195595,1.359899,0.046124,10.517418,32.426949,0.001883,0.156791,0.168617,1.063541,0.797712,0.498689
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.860000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,21.000000,0.995580,3.200000,0.550000,9.500000,5.000000,0.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,13.000000,37.000000,0.996740,3.310000,0.620000,10.200000,6.000000,1.000000
75%,9.300000,0.640000,0.430000,2.600000,0.091000,21.000000,60.000000,0.997860,3.400000,0.730000,11.100000,6.000000,1.000000
max,15.900000,1.580000,0.790000,15.400000,0.611000,72.000000,289.000000,1.003690,4.010000,1.980000,14.900000,8.000000,1.000000


# Преобразование исходных данных и построение моделей
## Стандартизация и переход к главным компонентам

В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент.

In [9]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 0:12].values)

# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
    np.around(frac_var_expl, 3),
    '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.261 0.186 0.141 0.099 0.083 0.055 0.051 0.043 0.035 0.027 0.015 0.005] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 44.7% разброса 12 объясняющих переменных.
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью `Pipeline` и оценим точность логистической регрессии с помощью перекрёстной проверки.

## Модель логистической регрессии с перекрёстной проверкой

In [10]:
# данные для обучения моделей
X_train = DF.iloc[:, 0:12] 
y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.802


In [11]:
X_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
694,9.0,0.470,0.31,2.70,0.084,24.0,125.0,0.99840,3.31,0.61,9.4,5
1300,5.2,0.645,0.00,2.15,0.080,15.0,28.0,0.99444,3.78,0.61,12.5,6
1351,7.2,0.620,0.01,2.30,0.065,8.0,46.0,0.99332,3.32,0.51,11.8,6
1246,7.4,0.740,0.07,1.70,0.086,15.0,48.0,0.99502,3.12,0.48,10.0,5
1206,7.2,0.360,0.46,2.10,0.074,24.0,44.0,0.99534,3.40,0.85,11.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1192,7.2,0.250,0.37,2.50,0.063,11.0,41.0,0.99439,3.52,0.80,12.4,7
319,9.6,0.770,0.12,2.90,0.082,30.0,74.0,0.99865,3.30,0.64,10.4,6
1531,6.1,0.705,0.10,2.80,0.081,13.0,28.0,0.99631,3.60,0.66,10.2,5
1248,6.9,0.410,0.33,2.20,0.081,22.0,36.0,0.99490,3.41,0.75,11.1,6


In [12]:
y_train

694     0
1300    1
1351    1
1246    0
1206    1
       ..
1192    1
319     1
1531    0
1248    1
614     1
Name: Y, Length: 1359, dtype: int64



```
X_train и y_train определены правильно.
```



## SVM с перекрёстной проверкой

Построим несколько вариантов модели SVM с различными ядерными функциями.

In [13]:
pipe_svc = make_pipeline(StandardScaler(), 
                         SVC(random_state = my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1] 
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, 
                  scoring = 'accuracy', refit = True, cv = kfold, 
                  n_jobs = -1)

# таймер
tic = time.perf_counter()

# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)

# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 6.66 секунд


In [14]:
# точность лучшей модели
np.around(gs.best_score_, 3)

1.0

Точность > 96%, изменялся лист param_range.

In [15]:
# параметры лучшей модели
# * ядерная функция 
gs.best_estimator_.get_params()['svc__kernel']

'linear'

In [16]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

0.01

In [17]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

'scale'

In [18]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

3

In [19]:
score.append(np.around(gs.best_score_,3))
score_models.append('sc_pca_svc')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


# Метод kNN

Реализуем метод k-ближайших соседей с преобразованием PCA.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [21]:
k_range = list(range(1,32))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 5)), ('knn', KNeighborsClassifier())])
param_grid = {
    'knn__n_neighbors': list(range(1,32))
}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 4.01 секунд


В ходе обучения модели я менял такие параметры как: n_splits (не дало особого увеличения точности) и n_components в PCA (дало прибавку к точности с 0.812 до 0.856)

In [22]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0



# Прогноз на отложенные наблюдения по лучшей модели

Ещё раз посмотрим на точность построенных моделей.  

In [25]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,sc_pca_logit,0.802
1,sc_pca_svc,1.000
2,sc_pca_knn,0.859


Все модели показывают хорошую точность по показателю $Acc$, при этом самой точной оказывается модель PCA + SVC. Сделаем прогноз на отложенные наблюдения.   

In [26]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat=gs.best_estimator_.predict(X=DF_predict.iloc[:, 0:12])
# точность# характеристики точности
print(classification_report(DF_predict.iloc[:,-1],y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       117
           1       1.00      1.00      1.00       123

    accuracy                           1.00       240
   macro avg       1.00      1.00      1.00       240
weighted avg       1.00      1.00      1.00       240



# Источники 

1. Джеймс Г., Уиттон Д., Хасти Т., Тибширани Р. Введение в статистическое обучение с примерами на языке R. Пер. с англ. С.Э. Мастицкого – М.: ДМК Пресс, 2016 – 450 с.  
1. *Рашка С.* Python и машинное обучение: крайне необходимое пособие по новейшей предсказательной аналитике, обязательное для более глубокого понимания методологии машинного обучения / пер. с англ. А.В. Логунова. – М.: ДМК Пресс, 2017. – 418 с.: ил.  
1. *Tong Wang*, *Cynthia Rudin*, *Finale Doshi-Velez*, *Yimin Liu*, *Erica Klampfl*, *Perry MacNeille* A Bayesian Framework for Learning Rule Sets for Interpretable Classification / Journal of Machine Learning Research 18 (2017) 1-37. URL: <https://jmlr.org/papers/volume18/16-003/16-003.pdf>  
1. *George Pipis* How to Run the Chi-Square Test in Python / medium.com. URL: <https://medium.com/swlh/how-to-run-chi-square-test-in-python-4e9f5d10249d>   
1. *Bernd Klein* What are Decision Trees? / python-course.eu. URL: <https://www.python-course.eu/Decision_Trees.php>  
1. Pruning decision trees - tutorial / kaggle.com. URL: <https://www.kaggle.com/arunmohan003/pruning-decision-trees-tutorial>  
1. Post pruning decision trees with cost complexity pruning / scikit-learn.org. URL: <https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html>  
1. *Piotr Płoński* Visualize a Decision Tree in 4 Ways with Scikit-Learn and Python / mljar.com. URL: <https://mljar.com/blog/visualize-decision-tree/>  
1. Random Forest Feature Importance Plot / www.analyseup.com. URL: <https://www.analyseup.com/learn-python-for-data-science/python-random-forest-feature-importance-plot.html>  